In [18]:
# question from  https://biginterview.com/blog/2014/09/sql-interview-questions.html  - #5

%load_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit

# Question 1 from : https://data36.com/sql-interview-questions-tech-screening-data-analysts/

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import csv
import pandas as pd

# had issues with tables persisting using in memory 
db_connection_string = 'sqlite:///temp_db.db'

Base = declarative_base()
engine = create_engine( db_connection_string )
Base.metadata.create_all(engine)

engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Engine(sqlite:///temp_db.db)

In [19]:
%sql sqlite:///temp_db.db

'Connected: @temp_db.db'

In [20]:


# load data into tables as is 
df = pd.read_csv('./data/interview/SQL Interview Samples - salesperson.csv')
df.to_sql(con=engine, name='salespeople', if_exists='replace')

df = pd.read_csv('./data/interview/SQL Interview Samples - customer.csv')
df.to_sql(con=engine, name='customers', if_exists='replace')

df = pd.read_csv('./data/interview/SQL Interview Samples - order.csv')
df.to_sql(con=engine, name='orders', if_exists='replace')


In [24]:
%%sql 

-- Given the tables above, write a query that will calculate the total commission by salesperson.

WITH order_with_sales AS ( 
    
    select * 
    FROM orders
    JOIN salespeople
    ON salespeople.id = orders.Sales_ID
)

select name, 
    SUM(Amount) as total_revenue_sold, 
    ( SUM(Amount) * (Commission_Rate/100.00)) as total_bonus
FROM order_with_sales
GROUP BY name
ORDER BY total_bonus DESC

 * sqlite:///temp_db.db
Done.


Name,total_revenue_sold,total_bonus
Pam,3126845,781711.25
Mark,2680766,321691.92
Alex,3017393,301739.3
John,3015672,180940.32
Amy,3217676,160883.80000000002


In [43]:
%%sql 

-- question #2
-- Name all salespersons who did not sell to company Red.

-- start off combining sales info to orders
WITH order_with_sales AS ( 
    
    select * 
    FROM orders
    JOIN salespeople
    ON salespeople.id = orders.Sales_ID
),


-- then add on the customer data
order_sales_customer AS (

    select customers.Name AS customer_name, order_with_sales.name as salesperson_name
    FROM order_with_sales
    JOIN customers
    ON order_with_sales.Cust_ID = customers.ID
    
)

-- count the rows and filter it out to only red, too much data to detect 0 
-- so we set a threshold and check that in having

select salesperson_name, customer_name , COUNT(*) as customer_total
from  order_sales_customer 
WHERE customer_name = 'Red' 
GROUP BY salesperson_name, customer_name
HAVING COUNT(*) <= 50

 * sqlite:///temp_db.db
Done.


salesperson_name,customer_name,customer_total
John,Red,47
Mark,Red,47


In [46]:
%%sql
--  official answer

SELECT s.name from salespeople s
WHERE s.ID NOT IN (
SELECT o.sales_id FROM orders o, customers c
WHERE o.cust_id = c.ID
AND c.Name = 'RED')

 * sqlite:///temp_db.db
Done.


Name
John
Amy
Mark
Pam
Alex
